# Recreation of "Paper Toss" Game Using Variable Air Resistance

## Introduction

_Paper Toss_ was an original game released on Android and IOS by Backflip Studios. The premise of this game is to "toss" a paper ball into a trashcan by swiping up on the touchscreen. The challanges arose when a fan was added to alter the trajectory of the ball. To compensate for the wind of the fan, players had to change the angle at which they would swipe in order to succesfully "make the basket". 

## Goal

The goal of this Jupyter Notebook is to fundamentally recreate _Paper Toss_ and create a _Ball Toss_ game using variable air resistance, characteristics of a baseball, and other physical relationships. 

## Assumptions

- Parameters used mirror a baseball rather than a paper ball
- The initial launch speed stays fixed
- The vertical launch angle stays fixed
- Inertial Drag is dominant (High Speeds)
- User must change horizontal launch angle to be able to score

## Theory 

To model the motion of a ball (sphere), we can numerically integrate these first-order differential equations using the RK4 integration technique:

$$\frac{dx}{dt} = v_x \qquad \frac{dy}{dt} = v_y \qquad \frac{dz}{dt} = v_z$$

$$\frac{d v_x}{dt} = \frac{F_{net,x}}{m} \qquad \frac{d v_y}{dt} = \frac{F_{net,y}}{m} \qquad \frac{d v_y}{dt} = \frac{F_{net,y}}{m}$$

where the net force is the sum of all forces acting on the ball. 

The *inertial* drag is:

$$\vec{F} = -\frac{1}{2}C_d \rho A v^2 \hat{v}$$

where $C_d$ is the drag coefficient, $\rho$ is the density of the fluid, and $A$ is the cross-sectional area of the object.

## Simulation

This program will be able to model the motion of a ball given a set of initial conditions and parameters. A scene will be created displaying the user the position of the ball, position of the hoop and the direction and strength of the wind.  There will be four different "situations" in which the wind will be randomly oriented in. This wind orientation will be decided every time the "main()" function is executed. The "main()" program will stop once the user selects the correct angle to succesfully score into the hoop. Note: Speeds used are very high and are just meant for gameplay realism, not to mimic the physical real world.


In [ ]:
## from vpython import *
import ode #ode.py should be in the same folder as your notebook
import numpy as np
import random as rand
import matplotlib.pyplot as plt

In [3]:
#parameters
g = 9.8 #N/kg
rho = 1.2 #kg/m^3
mu = 1.8e-5 #kg/m/s

hradius = 6

r = (7.5e-3)/2 #75 mm diameter
A = np.pi*r**2 #cross-sectional area
Cd = 0.5 #actually depends on speed
m = 0.05 #kg
b2 = 1/2*Cd*rho*A #will change as Cd changes


The 'vardrag' function handles the calculations of the new acceleration and velocity values for the ball. The Force of gravity and the variable inertial drag by the air and wind is taken into account.

In [4]:
def vardrag(d, t):
    """ Calculate and return the derivative of each quantity in an array d at the time t.
    
    Keyword arguments:
    t -- time at the beginning of the time step
    d -- an array of variables at time t
    """
    
    x = d[0]
    y = d[1]
    z = d[2]
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    dxdt = vx
    dydt = vy
    dzdt = vz
    
    ##Find Vrel
    vxrel = vx - vxwind
    vyrel = vy - vywind
    vzrel = vz - vzwind
    
    vrel = np.sqrt(vxrel**2 + vyrel**2 + vzrel**2)
    
    #drag
    a = 0.36
    b = 0.14
    c = 0.27
    vc = 34
    chi = (vrel-vc)/4
    
    if chi < 0:
        Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
    else:
        Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    
    Fdragx = -b2*vrel**2*vxrel/vrel
    Fdragy = -b2*vrel**2*vyrel/vrel
    Fdragz = -b2*vrel**2*vzrel/vrel
    
    #grav
    Fgravy = -m*g
    
    #Fnet
    Fnetx = Fdragx 
    Fnety = Fdragy + Fgravy
    Fnetz = Fdragz
    
    #derivative of velocity
    dvxdt = Fnetx/m
    dvydt = Fnety/m
    dvzdt = Fnetz/m
    
    derivs = np.array([dxdt, dydt, dzdt, dvxdt, dvydt, dvzdt])
    
    return derivs


The 'toss_calc' function requires the input of a horizontal angle in degrees. The function returns the spatial x, y and z coordinates of the ball as three separate lists: xlist, ylist, and zlist. 

In [5]:
def toss_calc(phideg):
    
    """ Calculate and return x, y and z position of the ball
    
    Keyword arguments:
    phideg -- horizontal launch angle in degrees
    """
    
    global b2, m, g, r, hradius

    
    #parameters
    g = 9.8 #N/kg
    rho = 1.2 #kg/m^3
    mu = 1.8e-5 #kg/m/s

    r = (7.5e-3)/2 #75 mm diameter
    A = np.pi*r**2 #cross-sectional area
    Cd = 0.5 #actually depends on speed
    m = 0.05 #kg
    b2 = 1/2*Cd*rho*A #will change as Cd changes

    # Intial launch speed
    vmag0 = 25

    # Vertical launch angle
    thetadeg = 60 #deg
    theta = thetadeg*np.pi/180 #convert deg to rad
    
    # Horizontal launch angle
    phi = phideg*np.pi/180 #convert deg to rad

    # Initial position components
    x = 0
    y = 0
    z = 0
    
    # Initial launch velocity components
    vx = vmag0*np.cos(theta)*np.sin(phi)
    vy = vmag0*np.sin(theta)
    vz = -vmag0*np.cos(theta)*np.cos(phi)

    t = 0
    h = 0.01

    # array for instantaneous position and velocity data
    data = np.array([x, y, z, vx, vy, vz])


    # lists for storing data to graph
    tlist = []
    xlist = []
    ylist = []
    zlist = []

    # store initial values
    tlist.append(t)
    xlist.append(x)
    ylist.append(y)
    zlist.append(z)

    while y >= 0:

        data = ode.RK4(vardrag, data, t, h)
        t = t + h

        x = data[0]
        y = data[1]
        z = data[2]
        tlist.append(t)
        xlist.append(x)
        ylist.append(y)
        zlist.append(z)
    
    return  xlist, ylist ,zlist

The 'getwind' function randomly selects the strength and direction of the wind which are pre-coded and tested. It is possible to 'score' with these four situations. Returns the components of the wind as an array.

In [6]:
def getwind():
    
    num = int(np.random.uniform(1,100))
    
    if(num > 0 and num <= 25):
        vwind = np.array([-80,0,0])
    
    elif(num > 26 and num <= 50):
        vwind = np.array([-120,0,-40])
        
    elif(num > 51 and num <= 75):
         vwind = np.array([0,0,0])
        
    elif(num > 76 and num <= 100):
         vwind = np.array([60,0,10])
            
    return vwind

The 'main' function creates the user interface and handles the framework for the actual game. Scene is displayed and allows for user interaction. The game continues to run until the user 'scores'.

In [ ]:
def main():
    
    global vxwind, vywind, vzwind
    
    scene = canvas(title="Ball Toss - Instructions: Look at blue arrow for direction and strength of the wind. Change angle to get ball inside of hoop. Click 'Play' to start.")
    
    warray = getwind()
    
    vxwind = warray[0]
    vywind = warray[1]
    vzwind = warray[2]
    vwind = vec(vxwind,vywind,vzwind)
    
    pointer = arrow(pos=vector(0,0,0), axis=.15*vwind, shaftwidth=1, color=color.blue)
    ball = sphere(pos = vec(0,0,0), radius = r*400, color = color.white)
    b = attach_trail(ball)
    hoop = ring(pos=vector(0,15/2,-50), axis=vector(0,1,0), radius = hradius, thickness=0.05*hradius, color=color.orange)
    
    goal = False
    
    while(goal == False):
        
        scene.pause()
        val = int(input("Enter your angle in degrees: "))
        
        ## Calculates the motion of the ball with a given angle phi.
        xl, yl, zl = toss_calc(val)
        sizelist = np.size(zl)
        
        ## Animates the ball using the previously calculated position values
        for n in range(1,sizelist):
            rate(100)
            ball.pos = vec(xl[n],yl[n],zl[n]) 
            dist = np.sqrt((xl[n]-hoop.pos.x)**2+(zl[n]-hoop.pos.z)**2)
            
        ## Checks to see if ball is within hoop
            if(dist<hradius and yl[n] > hoop.pos.y-1):
                goal = True
                
        if(goal == True):
            print("GOOAAALLLLL!!!!")
            return

        print("Try again!")
        b.clear()
        ball.pos = vec(0,0,0) 
    return

  Here is when the 'main()' function is finally called. To interact with the game, the play button must be clicked. Once the user enters the horizontal angle, the ball will be launched accordingly. If the user misses the hoop, "Try Again!" will be displayed. The user must click the play button once again. Steps are repeated until the user scores.

In [ ]:
main()

## Conclusion

This adaptation of _Paper Toss_ seems to work as expected. When experimenting with different wind parameters, it became apparent that only a few situations were usable. If the wind was too strong, sometimes it would be impossible to score into the hoop. The angle would be too large and the ball would fall short of the target. To overcome this issue, the initial launch speed would have to increased. For our purposes, the initial launch speed was kept fixed, but doing so creates a limiting factor within the game. Something else that could be explored in the future is using linear (viscous) drag rather than inertial drag and comparing the game behavior and difficulty between the two.  